## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [1]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')
df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [2]:
# View the column names in the data
print(df.columns)


Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')


In [3]:
# Use the describe function to gather some basic statistics
df.describe()

,client_id,order_id,order_week,order_year,unit_price,unit_cost,unit_weight,qty,line_number
count,54639.000000,5.463900e+04,54639.000000,54639.000000,54639.000000,54639.000000,54639.000000,5.463900e+04,54639.000000
mean,54837.869416,5.470190e+06,11.359139,2022.993064,136.267207,99.446073,5.004116,5.702646e+02,2.979667
std,25487.438231,2.599807e+06,7.023499,0.082997,183.873135,133.164267,5.326599,1.879552e+04,2.436320
min,10033.000000,1.000886e+06,1.000000,2022.000000,0.010000,0.010000,0.000000,0.000000e+00,0.000000
25%,33593.000000,3.196372e+06,6.000000,2023.000000,20.800000,14.840000,1.450000,3.200000e+01,1.000000
50%,53305.000000,5.496966e+06,11.000000,2023.000000,68.310000,49.890000,3.240000,6.800000e+01,3.000000
75%,78498.000000,7.733869e+06,17.000000,2023.000000,173.160000,125.570000,6.890000,1.700000e+02,5.000000
max,99984.000000,9.998480e+06,52.000000,2023.000000,1396.230000,846.270000,46.430000,3.958244e+06,9.000000


In [4]:
# Use this space to do any additional research
# and familiarize yourself with the data.
df['order_date'] = pd.to_datetime(df['order_date'])
unit_weight = df.groupby('order_year')['unit_weight'].sum()
print(unit_weight)

order_year
2022      1829.85
2023    271590.06
Name: unit_weight, dtype: float64


In [5]:
# What three item categories had the most entries?
top_three_categories=df['category'].value_counts().head(3)
top_three_categories 




category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64

In [6]:
# For the category with the most entries, which subcategory had the most entries?
subcategory = df.loc[df['category'] == 'consumables', 'subcategory'].value_counts()
subcategory
# list only the most entry for consumables

subcategory
bathroom supplies    6424
paper products       5874
kitchen supplies     5746
misc                 2891
pens                 2603
Name: count, dtype: int64

In [7]:
# Which five clients had the most entries in the data?
top_five_client_id=df['client_id'].value_counts().head(5)
top_five_client_id

client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64

In [8]:
# Store the client ids of those top 5 clients in a list.
top_five_client_id = [33615, 66037, 46820, 38378, 24741]
top_five_client_id

[33615, 66037, 46820, 38378, 24741]

In [9]:
# How many total units (the qty column) did the client with the most entries  order?

most_entries_client_id = df['client_id'].value_counts().idxmax()
most_entries_client_data = df[df['client_id'] == most_entries_client_id]
total_units_ordered = most_entries_client_data['qty'].sum()
print("Total units ordered by the client with the most entries:", total_units_ordered)




Total units ordered by the client with the most entries: 64313


## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [10]:
# Create a column that calculates the subtotal for each line using the unit_price and the qty
df['line_subtotal'] = df['unit_price'] * df['qty']
df[['unit_price','qty','line_subtotal']].head(2)



,unit_price,qty,line_subtotal
0,1096.80,105,115164.00
1,24.95,21,523.95


In [11]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound for orders over 50 pounds and $10 per pound for items 50 pounds or under.

df['total_weight'] = df['qty'] * df['unit_weight']
#shipping_price = df[['unit_price', 'unit_weight', 'qty', 'total_weight', 'shipping_price']]

def shipping_calculator(weight):
    if weight > 50:
        return weight * 7
    return weight * 10

#df['total_weight'] = df['qty'] * df['unit_weight']
df['shipping_price'] = df['total_weight'].apply(shipping_calculator)
shipping_price = df[['unit_price', 'unit_weight', 'qty', 'total_weight', 'shipping_price']]
shipping_price.head(3)




,unit_price,unit_weight,qty,total_weight,shipping_price
0,1096.80,7.50,105,787.50,5512.50
1,24.95,1.49,21,31.29,312.90
2,13.52,1.68,39,65.52,458.64


In [13]:
# Create a column for the total price using the subtotal and the shipping price along with a sales tax of 9.25%

df['total_price'] = df['line_subtotal'] + df['shipping_price'] 
                                                                                                                                                         
sales_tax_rate =  0.0925  # 9.25% expressed as a decimal

df['total_price'] = df['line-subtotal'] + \
    df['shipping_price'] + \
        (df['line_subtotal'] *df['sales_tax_rate'])
df['line_subtotal'] = round(df['total_price'], ndigits=2)
#total_price = [['subtotal', 'shipping_price', 'line-price',]]
total_price = [['line_subtotal', 'shipping_price', 'line-price',]]

total_price.head(3)



KeyError: 'line-subtotal'

In [15]:
import pandas as pd

# Example data (replace with your actual data or loading mechanism)
data = {
    'line_subtotal': [100, 150, 200],
    'shipping_price': [10, 15, 20]
}

# Create DataFrame
df = pd.DataFrame(data)

# Check column names
print("Columns in DataFrame:")
print(df.columns)

# Example calculation with existing columns
sales_tax_rate = 0.0925
df['total_price'] = (df['line_subtotal'] + df['shipping_price']) * (1 + sales_tax_rate)
df['total_price_round'] = df['total_price'].round(2)

# Print updated DataFrame
print("\nUpdated DataFrame:")
print(df)


Columns in DataFrame:
Index(['line_subtotal', 'shipping_price'], dtype='object')

Updated DataFrame:
   line_subtotal  shipping_price  total_price  total_price_round
0            100              10     120.1750             120.18
1            150              15     180.2625             180.26
2            200              20     240.3500             240.35


In [ ]:
print(df)


       unit_price  unit_weight   qty  total_weight  shipping_price
0         1096.80         7.50   105        787.50         5512.50
1           24.95         1.49    21         31.29          312.90
2           13.52         1.68    39         65.52          458.64
3           36.42         1.23    29         35.67          356.70
4          195.10        46.43    20        928.60         6500.20
...           ...          ...   ...           ...             ...
54634       83.13         2.25    33         74.25          519.75
54635      206.59        11.70    47        549.90         3849.30
54636       65.66         4.16   475       1976.00        13832.00
54637        1.48        18.04   112       2020.48        14143.36
54638        3.01         2.07  1031       2134.17        14939.19

[54639 rows x 5 columns]


In [ ]:
round(.12345, ndigits=4)

0.1235

In [ ]:
?df.apply

Signature:
df.apply(
    func: 'AggFuncType',
    axis: 'Axis' = 0,
    raw: 'bool' = False,
    result_type: "Literal['expand', 'reduce', 'broadcast'] | None" = None,
    args=(),
    by_row: "Literal[False, 'compat']" = 'compat',
    **kwargs,
)
Docstring:
Apply a function along an axis of the DataFrame.

Objects passed to the function are Series objects whose index is
either the DataFrame's index (``axis=0``) or the DataFrame's columns
(``axis=1``). By default (``result_type=None``), the final return type
is inferred from the return type of the applied function. Otherwise,
it depends on the `result_type` argument.

Parameters
----------
func : function
    Function to apply to each column or row.
axis : {0 or 'index', 1 or 'columns'}, default 0
    Axis along which the function is applied:

    * 0 or 'index': apply function to each column.
    * 1 or 'columns': apply function to each row.

raw : bool, default False
    Determines if row or column is passed as a Series or ndarray ob

In [ ]:
dir(df)

['T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__dataframe__',
 '__dataframe_consortium_standard__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pandas_priority__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '

In [ ]:
# Create a column for the cost of each line using unit cost, qty, and
# shipping price (assume the shipping cost is exactly what is charged to the client).



In [ ]:
# Create a column for the profit of each line using line cost and line price



## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [ ]:
# Check your work using the totals above
2742071,2023-02-27,9,2023,ACU90518-83-2B,consumables,bathroom supplies,5.59,3.05,10.46,77,5

2173913,2023-02-19,7,2023,XED42806-14-3M,consumables,paper products,3.28,2.09,1.04,36,2

6128929,2023-03-12,10,2023,UUX03397-27-4C,consumables,misc,107.61,98.35,6.28,1089,9

## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [ ]:
# How much did each of the top 5 clients by quantity spend? Check your work from Part 1 for client ids.
'top_five_client'


'top_five_client'

In [ ]:
# Create a summary DataFrame showing the totals for the for the top 5 clients with the following information:
# total units purchased, total shipping price, total revenue, and total profit. 


In [ ]:
# Format the data and rename the columns to names suitable for presentation.

# Define the money columns. 


# Define a function that converts a dollar amount to millions.


# Apply the currency_format_millions function to only the money columns. 


# Rename the columns to reflect the change in the money format. 




In [ ]:
# Sort the updated data by "Total Profit (millions)" form highest to lowest and assign the sort to a new DatFrame.
